In [ ]:
#pip install yfinance

In [1]:
import pandas as pd
from statsmodels.tsa.api import VAR
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import yfinance as yf

In [3]:
stk_data = yf.download("RELIANCE.NS", start="2013-06-01", end="2022-02-11")
stk_data

C:\Users\admin\AppData\Local\Temp\ipykernel_20620\4010520275.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stk_data = yf.download("RELIANCE.NS", start="2013-06-01", end="2022-02-11")
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS
Date,,,,,
2013-06-03,167.620468,171.332833,167.056686,170.981802,14165128
2013-06-04,165.982315,170.152093,165.758941,168.194853,14752690
2013-06-05,170.620163,171.109473,166.046170,166.046170,12748842
2013-06-06,168.492722,171.024366,167.663032,170.194677,17113393
2013-06-07,166.705673,170.620152,166.152542,168.960745,9420701
...,...,...,...,...,...
2022-02-04,1065.006226,1072.840921,1060.346589,1069.437484,11061241
2022-02-07,1058.519165,1076.655277,1052.991451,1069.894201,10714467


In [4]:
stk_data=stk_data[["Open","High","Low","Close"]]

In [7]:
stk_data

Price,Open,High,Low,Close
Ticker,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS
Date,,,,
2013-06-03,170.981802,171.332833,167.056686,167.620468
2013-06-04,168.194853,170.152093,165.758941,165.982315
2013-06-05,166.046170,171.109473,166.046170,170.620163
2013-06-06,170.194677,171.024366,167.663032,168.492722
2013-06-07,168.960745,170.620152,166.152542,166.705673
...,...,...,...,...
2022-02-04,1069.437484,1072.840921,1060.346589,1065.006226
2022-02-07,1069.894201,1076.655277,1052.991451,1058.519165


In [9]:
#performing Minmax scaler
from sklearn.preprocessing import MinMaxScaler
Ms = MinMaxScaler()
data1= Ms.fit_transform(stk_data)
print("Len:",data1.shape)

Len: (2144, 4)


In [11]:
data1=pd.DataFrame(data1,columns=["Open","High","Low","Close"])

In [13]:
data1

,Open,High,Low,Close
0,0.004541,0.002707,0.004227,0.001514
1,0.001977,0.001622,0.003019,0.000000
2,0.000000,0.002502,0.003286,0.004286
3,0.003817,0.002424,0.004791,0.002320
4,0.002681,0.002052,0.003385,0.000669
...,...,...,...,...
2139,0.831146,0.830910,0.835520,0.830886
2140,0.831566,0.834414,0.828675,0.824890
2141,0.826796,0.835757,0.830801,0.841335
2142,0.843818,0.847173,0.851207,0.852925


In [15]:
#Split train and test set
training_size = round(len(data1 ) * 0.80)
print(training_size)
X_train=data1[:training_size]
X_test=data1[training_size:]
print("X_train length:",X_train.shape)
print("X_test length:",X_test.shape)
y_train=data1[:training_size]
y_test=data1[training_size:]
print("y_train length:",y_train.shape)
print("y_test length:",y_test.shape)

1715
X_train length: (1715, 4)
X_test length: (429, 4)
y_train length: (1715, 4)
y_test length: (429, 4)


In [17]:
import warnings
warnings.filterwarnings("ignore")

In [19]:
# Define combinations manually
combinations = [['Close', 'High'],['Close', 'High', 'Open'],['Close', 'High', 'Open', 'Low']]

# Empty list to store results
performance = {"Model": [],"RMSE": [],"MaPe": [],"Lag": [],"Test": []}


In [23]:

test_obs = 28

# Loop through combinations
for cols in combinations:
    dataset = data1[cols]
    train = dataset[:-test_obs]
    test = dataset[-test_obs:]
    
    model = VAR(train)
    for i in [1,2,3,4,5,6,7,8,9,10]:
        
        results = model.fit(i)
        print('Order =', i)
        print('AIC: ', results.aic)
        print('BIC: ', results.bic)
        print()

   # model = VAR(train)

    # Select best lag order using AIC
    lag_selection = model.select_order(maxlags=12)
    order = lag_selection.selected_orders['aic']

    # Fit and forecast
    result = model.fit(order)
  
    lagged_values = train.values[-order:]
    pred = result.forecast(y=lagged_values, steps=test_obs)
    pred_df = pd.DataFrame(pred, columns=cols)

    pred_df.to_csv("varforecasted_{}.csv".format(test_obs))

    # Calculate metrics
    rmse = round(mean_squared_error(test, pred_df, squared=False), 4)
    mape = round(mean_absolute_percentage_error(test, pred_df), 6)

    # Append to performance
    performance["Model"].append(cols)
    performance["RMSE"].append(rmse)
    performance["MaPe"].append(mape)
    performance["Lag"].append(order)
    performance["Test"].append(test_obs)

# Convert to DataFrame and print
perf_df = pd.DataFrame(performance)
print(perf_df)


Order = 1
AIC:  -20.470046135241066
BIC:  -20.453998447038288

Order = 2
AIC:  -20.50752158328187
BIC:  -20.480765021472834

Order = 3
AIC:  -20.54177130804475
BIC:  -20.50429752846755

Order = 4
AIC:  -20.55683091291605
BIC:  -20.50863156082946

Order = 5
AIC:  -20.558099085547873
BIC:  -20.499165795612928

Order = 6
AIC:  -20.560263684981024
BIC:  -20.490588081242528

Order = 7
AIC:  -20.56166456189784
BIC:  -20.481238257765852

Order = 8
AIC:  -20.57768444836259
BIC:  -20.48649904659386

Order = 9
AIC:  -20.58363068334362
BIC:  -20.481677776022995

Order = 10
AIC:  -20.583042805142927
BIC:  -20.470313973664698

Order = 1
AIC:  -31.8495972922051
BIC:  -31.817501915799536

Order = 2
AIC:  -31.90337768867034
BIC:  -31.847188908871363

Order = 3
AIC:  -31.935652581144655
BIC:  -31.855351624907794

Order = 4
AIC:  -31.94714314761229
BIC:  -31.842711218091353

Order = 5
AIC:  -31.95483162463499
BIC:  -31.82624990114057

Order = 6
AIC:  -31.959949144217166
BIC:  -31.80719878217508

Order =